# Interest Rate Risk II: Convexity

In [1]:
import numpy as np
import pandas as pd
!pip install numpy_financial
import numpy_financial as npf
import plotly.graph_objects as go


[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
def bondpv(cr,ytm,nop,ttm, principal):
    ''' 
    Inputs:
        cr:             coupon rate (per year, in decimal)
        ytm:            yield-to-maturity (per year, in decimal)
        nop:            number of payments per year
        ttm:            time-to-maturity (in periods)
        principal:      face value
    Outputs:
        p:              bond price
    '''
    dr = ytm/nop
    c = cr/nop*principal
    p = -npf.pv(dr,ttm,c,principal)
    return p

def mduration(cr,ytm,nop,ttm, principal):
    ''' 
    Inputs:
        cr:             coupon rate (per year, in decimal)
        ytm:            yield-to-maturity (per year, in decimal)
        nop:            number of payments per year
        ttm:            time-to-maturity (in periods)
        principal:      face value
    Outputs:
        duration:       modified duration
    '''
    dr = ytm/nop
    c = cr/nop*principal
    p = -npf.pv(dr,ttm,c,principal)
    # Write out CFs and discount
    cfs = c*np.ones(ttm)
    cfs[-1] = cfs[-1]+principal
    pvs = cfs/(1+dr)**(1+np.arange(ttm))
    # Calculate weights
    wgts = pvs / np.sum(pvs)
    t = (1+np.arange(ttm))/nop
    # Calculate duration
    duration = t @ wgts
    return duration / (1+dr)
def implied_pct_chg(cr,ytm,nop,ttm, principal, delta_ytm):
    dmod = mduration(cr,ytm,nop,ttm, principal)
    return -dmod * delta_ytm
def actual_pct_chg(cr,ytm,nop,ttm, principal, delta_ytm):
    p = bondpv(cr,ytm,nop,ttm, principal)
    pnew = bondpv(cr,ytm+delta_ytm,nop,ttm, principal)
    return pnew/p - 1

### From last time: duration appoximation

In [3]:
CR  = 0.05          # annual coupon rate
YTM = 0.10          # annual yield to maturity
NOP = 1             # payments per year
PRINCIPAL = 100     # face value of the bond
TTM_LIST  = [10, 20]
DELTA_YIELDS    = np.arange(-0.02,0.025,0.005)
column_index = pd.MultiIndex.from_product([['actual_pct_chg','approx_pct_chg'],TTM_LIST])
df = pd.DataFrame(dtype=float,columns=column_index,index=DELTA_YIELDS)
df.index.name='delta_ytm'

df['new_ytm'] = YTM + df.index
for dy in df.index:
    for ttm in TTM_LIST:
        df.loc[dy,('actual_pct_chg',ttm)] = actual_pct_chg(CR,YTM,NOP,ttm,PRINCIPAL,dy)
        df.loc[dy,('approx_pct_chg',ttm)] = implied_pct_chg(CR,YTM,NOP,ttm,PRINCIPAL,dy)
pd.options.display.float_format = '{:,.4f}'.format        
df

actual_pct_chg         approx_pct_chg         new_ytm
                      10      20             10      20        
delta_ytm                                                      
-0.0200           0.1529  0.2283         0.1393  0.1953  0.0800
-0.0150           0.1120  0.1645         0.1045  0.1465  0.0850
-0.0100           0.0729  0.1054         0.0696  0.0976  0.0900
-0.0050           0.0356  0.0507         0.0348  0.0488  0.0950
0.0000            0.0000  0.0000        -0.0000 -0.0000  0.1000
0.0050           -0.0340 -0.0470        -0.0348 -0.0488  0.1050
0.0100           -0.0666 -0.0908        -0.0696 -0.0976  0.1100
0.0150           -0.0977 -0.1314        -0.1045 -0.1465  0.1150
0.0200           -0.1274 -0.1692        -0.1393 -0.1953  0.1200

## Convexity

In [4]:
def convexity(cr,ytm,nop,ttm, principal):
    ''' 
    Inputs:
        cr:             coupon rate (per year, in decimal)
        ytm:            yield-to-maturity (per year, in decimal)
        nop:            number of payments per year
        ttm:            time-to-maturity (in periods)
        principal:      face value
    Outputs:
        convexity:      convexity of the bond
    '''
    dr = ytm/nop
    c = cr/nop*principal
    p = -npf.pv(dr,ttm,c,principal)
    # Write out CFs and discount
    cfs = c*np.ones(ttm)
    cfs[-1] = cfs[-1]+principal
    pvs = cfs/(1+dr)**(1+np.arange(ttm))
    # Calculate weights
    wgts = pvs / np.sum(pvs)
    t_terms = (1+np.arange(ttm)) * (2+np.arange(ttm)) / nop**2
    # Calculate duration
    convexity = t_terms @ wgts / (1+dr)**2
    return convexity

In [5]:
TTM_PERIODS = TTM_LIST[0]
convexity(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL)

63.39892288902894

In [6]:
mduration(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL)   

6.9644205086450865

### Approximate price change with convexity

For a change in yield $y$, the percent change in price is:
$$\frac{\Delta P}{P} \approx -D_{\text{modified}} \cdot \Delta y + 0.5\cdot\text{convexity}\cdot (\Delta y)^2.$$

In [7]:
def implied_pct_chg2(cr,ytm,nop,ttm, principal, delta_ytm):
    dmod = mduration(cr,ytm,nop,ttm, principal)
    convx= convexity(cr,ytm,nop,ttm, principal)
    return -dmod * delta_ytm + 0.5*convx*delta_ytm**2

In [8]:
implied_pct_chg(0.06,0.05,1,5,100,-0.015)

0.0639678749012571

In [9]:
actual_pct_chg(0.06,0.05,1,5,100,-0.015)

0.06669403978717336

In [10]:
implied_pct_chg2(0.06,0.05,1,5,100,-0.015)

0.06660533509198213

### Testing the second-order approximation

In [11]:
# Initialize new columns
for ttm in TTM_LIST:
    df[('approx2_pct_chg',ttm)] = np.nan
for dy in df.index:
    for ttm in TTM_LIST:
        df.loc[dy,('approx2_pct_chg',ttm)] = implied_pct_chg2(CR,YTM,NOP,ttm,PRINCIPAL,dy)


In [12]:
df

actual_pct_chg         approx_pct_chg         new_ytm  \
                      10      20             10      20           
delta_ytm                                                         
-0.0200           0.1529  0.2283         0.1393  0.1953  0.0800   
-0.0150           0.1120  0.1645         0.1045  0.1465  0.0850   
-0.0100           0.0729  0.1054         0.0696  0.0976  0.0900   
-0.0050           0.0356  0.0507         0.0348  0.0488  0.0950   
0.0000            0.0000  0.0000        -0.0000 -0.0000  0.1000   
0.0050           -0.0340 -0.0470        -0.0348 -0.0488  0.1050   
0.0100           -0.0666 -0.0908        -0.0696 -0.0976  0.1100   
0.0150           -0.0977 -0.1314        -0.1045 -0.1465  0.1150   
0.0200           -0.1274 -0.1692        -0.1393 -0.1953  0.1200   

          approx2_pct_chg          
                       10      20  
delta_ytm                          
-0.0200            0.1520  0.2245  
-0.0150            0.1116  0.1629  
-0.0100            0.0728  0.1050  
-0.0050            0.0356  0.0506  
0.0000            -0.0000 -0.0000  
0.0050            -0.0340 -0.0470  
0.0100            -0.0665 -0.0903  
0.0150            -0.0973 -0.1300  
0.0200            -0.1266 -0.1661

In [13]:
# Plot the data
ttm = TTM_LIST[1]

fig = go.Figure()
ttm_years = int(ttm/NOP)
trace_act=go.Scatter(x=df.index, y=df[('actual_pct_chg',ttm)], mode='lines', name=str(ttm_years)+' years to maturity: Actual % Change')
trace_approx=go.Scatter(x=df.index, y=df[('approx_pct_chg',ttm)], mode='lines', name=str(ttm_years)+' years to maturity: 1st order Approx % Change')
trace_approx2=go.Scatter(x=df.index, y=df[('approx2_pct_chg',ttm)], mode='lines', name=str(ttm_years)+' years to maturity: 2nd order Approx % Change')
fig.add_trace(trace_act)
fig.add_trace(trace_approx)
fig.add_trace(trace_approx2)

fig.update_xaxes(title='Change in Yield',tickformat=".2%")
fig.update_yaxes(title='% Change in Price',tickformat=".2%")
fig.update_layout(title='Interest Rate Risk')
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.6))
fig.show()

### Estimating Duration and Convexity


In [14]:
# Assume we observe prices at increments of DY (in yield difference)
DY = 0.0005

p0 = bondpv(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL)
pplus  = bondpv(CR,YTM+DY,NOP,TTM_PERIODS,PRINCIPAL)
pminus = bondpv(CR,YTM-DY,NOP,TTM_PERIODS,PRINCIPAL)


In [15]:
def est_mduration(dy, p0, pplus, pminus):
    return (pminus - pplus)/ (p0*2*dy)
def est_convexity(dy, p0, pplus, pminus):
    return (pplus + pminus - 2*p0)/ (p0*(dy**2))


In [16]:
print(f'Actual modified duration is:\t {mduration(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL):,.4f}')
print(f'Estimated modified duration is:\t {est_mduration(DY, p0, pplus, pminus):,.4f}')

Actual modified duration is:	 6.9644
Estimated modified duration is:	 6.9644


In [17]:
print(f'Actual convexity is:\t {convexity(CR,YTM,NOP,TTM_PERIODS,PRINCIPAL):,.4f}')
print(f'Estimated convexity is:\t {est_convexity(DY, p0, pplus, pminus):,.4f}')


Actual convexity is:	 63.3989
Estimated convexity is:	 63.3991
